#**LSTM Model menggunakan PyTorch : DeteksiSarkasme.json Dataset🔖**

In [29]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch


In [30]:
# Check if GPU or TPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [31]:
# Step 1: Load the dataset
import json

# Load the dataset (replace the path if needed)
with open("/content/Cleaned_DeteksiSarkasme.json", "r") as file:
    data = json.load(file)

In [32]:
# Step 2: Preprocess the dataset
df = pd.DataFrame(data)


In [33]:
# Extract the headlines and labels
X = df['headline'].values
y = df['is_sarcastic'].values

In [22]:
# Step 2.1: Preprocess the text data (Remove punctuation, lowercase, etc.)
import re
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [34]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


In [36]:
# Tokenize the input text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)


In [37]:
# Prepare the dataset
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)


In [38]:
# Convert labels to tensor format
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)


In [39]:
# Create Dataset for Hugging Face Trainer
from torch.utils.data import Dataset

class SarcasmDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [40]:
train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

In [41]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [43]:
# Train the model
trainer.train()

In [44]:
# Evaluate the model
results = trainer.evaluate()
print(results)